In [1]:
import fitz  # pip install pymupdf

with fitz.open("../data/sample.pdf") as doc:
    text = ""
    for page in doc:
        text += page.get_text()
    lst = text.split('\n')
    result = []
    for string in lst:
        for item in string.split(':'):
            result.append(item.strip())

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from os import walk
from datetime import datetime

path = '../data/DATA_xml.09.04'
file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)
file_names = [x for x in file_names if x.endswith('.xml')]

DELIMITER = ' > '

data = {
    'Ký hiệu mẫu hóa đơn':          ['templateCode',          '',           'FORM_NO',                          'InvoicePattern',   'MauSo'],
    'Ký hiệu hoá đơn':              ['invoiceSeries',         'KHHDon',     'SERIAL_FORM',                      'SerialNo',         'Kyhieu'],
    'Số hoá đơn':                   ['invoiceNumber',         'SHDon',      'SERI_NO',                          'InvoiceNo',        'SoHoaDon'],
    'Ngày, tháng, năm lập hóa đơn': ['invoiceIssuedDate',     'NLap',       'RECEIPTDATE',                      'ArisingDate',      'NgayHoaDon'],
    'Tên người bán':                ['sellerLegalName',       'Ten',        'SELLER',                           'ComName',          'TenNguoiBan'],
    'Doanh số mua chưa có thuế':    ['totalAmountWithoutVAT', 'TgTCThue',   'GETTICKET' + DELIMITER + 'AMOUNT', 'Total',            'TongTienTruocThue'],
    'Thuế suất':                    ['vatPercentage',         'TSuat',      '',                                 'VAT_Rate',         ''],
    'Tiền thuế GTGT HĐ mua vào':    ['totalVATAmount',        'TgTThue',    'GETVAT' + DELIMITER + 'VN_VAT',    'VAT_Amount',       'TongTienThue'],
    'Tổng cộng':                    ['totalAmountWithVAT',    'TgTTTBSo',   'SALETOTAL' + DELIMITER + 'VN_SUM', 'Amount',           'TongTien'],
}

# index = _type
datetime_format = (
    ('%Y-%m-%d', '%Y/%m/%d'), '%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y', '%Y-%m-%d'
)

MAX_TYPE = len(data['Ký hiệu mẫu hóa đơn'])


def get_value(soup, key, _type):
    if(DELIMITER in data[key][_type]):
        return soup.find(
                data[key][_type].split(DELIMITER)[0]
            ).find(
                data[key][_type].split(DELIMITER)[1]
            ).get_text()
    else:
        return soup.find(
            data[key][_type]).get_text()


result_list = []
for fn in file_names:
    result_dict = {}
    _type = 0
    while result_dict == {} and _type < MAX_TYPE:
        with open(fn, 'rb') as f:
            soup = BeautifulSoup(f, features="xml")
            
            for key in data.keys():
                
                try:
                    value = get_value(soup, key, _type)
                except AttributeError:
                    continue

                if key == 'Ngày, tháng, năm lập hóa đơn':
                    result_dict['date_origin'] = value
                    value = value[:10].replace(' ','')
                    if isinstance(datetime_format[_type], tuple):
                        for fmt in datetime_format[_type]:
                            try:
                                value = datetime.strptime(value, fmt)
                                break
                            except ValueError:
                                continue
                    else:
                        value = datetime.strptime(value, datetime_format[_type])

                result_dict[key] = value
               
            if _type == 2 and result_dict:
                (result_dict['Tổng cộng']) = result_dict['Tổng cộng'].replace('.', '')
                (result_dict['Doanh số mua chưa có thuế']) = result_dict['Doanh số mua chưa có thuế'].replace('.', '')
                (result_dict['Tiền thuế GTGT HĐ mua vào']) = result_dict['Tiền thuế GTGT HĐ mua vào'].replace('.', '')
        _type += 1

    if not result_dict:
        print('can not get data from: ' + fn)
    else:
        result_list.append(result_dict)


df = pd.DataFrame.from_records(result_list)


df['Tiền thuế GTGT HĐ mua vào'] = df['Tiền thuế GTGT HĐ mua vào'].astype(float)
df['Doanh số mua chưa có thuế'] = df['Doanh số mua chưa có thuế'].astype(float)
df['Tổng cộng'] = df['Tổng cộng'].astype(float)
df['Ngày, tháng, năm lập hóa đơn'] = df['Ngày, tháng, năm lập hóa đơn'].dt.strftime('%Y-%m-%d')

df.to_excel('result.xlsx', index=False)
